In [23]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.io.input import loadMidiFile
from src.io.output import pianoRoll
from src.adaptation import Adaptation
from src.datatypes.melody_data import MelodyData
from definitions import SequenceType

In [24]:
a = Adaptation()
a.available_operations

[src.adaptation.operations.start_end_on_ct_operation.StartEncOnCTOperation,
 src.adaptation.operations.transpose_notes_operation.TransposeNotesOperation,
 src.adaptation.operations.transpose_sequence_operation.TransposeSequenceOperation]

In [25]:
for op in a.available_operations:
    print(op.__name__)
    print(op.__doc__)

StartEncOnCTOperation
None
TransposeNotesOperation
None
TransposeSequenceOperation
Estimates the key of both sequences and transposes the base sequence to be in the same key as the control sequence.


In [26]:
a.construct_pipeline(a.available_operations)
a.pipeline.get_operations()

['StartEncOnCTOperation',
 'TransposeNotesOperation',
 'TransposeSequenceOperation']

In [27]:
a.construct_pipeline(['TransposeSequenceOperation'])
a.pipeline.get_operations()

['TransposeSequenceOperation']

In [28]:
midi1 = loadMidiFile('../midi/examples/monophonic/bohemian_mama2_4b.mid')
midi2 = loadMidiFile('../midi/examples/monophonic/africa_intro_2b.mid')

mel1 = MelodyData(midi1, SequenceType.REC_INPUT, {})
mel2 = MelodyData(midi2, SequenceType.GEN_BASE, {})

mel2.sequence.instruments[0].notes

[Note(start=0.000000, end=0.260417, pitch=57, velocity=77),
 Note(start=0.354167, end=0.401042, pitch=57, velocity=77),
 Note(start=0.604167, end=0.645833, pitch=57, velocity=77),
 Note(start=0.854167, end=0.906250, pitch=57, velocity=77),
 Note(start=0.979167, end=1.052083, pitch=57, velocity=77),
 Note(start=1.229167, end=1.296875, pitch=56, velocity=77),
 Note(start=1.759375, end=1.926042, pitch=78, velocity=98),
 Note(start=1.884375, end=2.019792, pitch=76, velocity=90),
 Note(start=2.009375, end=2.155208, pitch=73, velocity=82),
 Note(start=2.134375, end=2.254167, pitch=76, velocity=86),
 Note(start=2.259375, end=2.431250, pitch=78, velocity=94),
 Note(start=2.389583, end=2.519792, pitch=76, velocity=86),
 Note(start=2.509375, end=2.603125, pitch=73, velocity=86),
 Note(start=2.634375, end=2.759375, pitch=76, velocity=90),
 Note(start=2.759375, end=2.946875, pitch=78, velocity=94),
 Note(start=2.884375, end=3.004167, pitch=76, velocity=86),
 Note(start=3.014583, end=3.139583, pitc

In [29]:
ad_mel2 = mel2.to_adaptation_data()
ad_mel2.sequence.show('text')

{0.0} <music21.stream.Part 0x16366fa2108>
    {0.0} <music21.tempo.MetronomeMark animato Quarter=120.0>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Note A>
    {0.7083} <music21.note.Note A>
    {1.2083} <music21.note.Note A>
    {1.7083} <music21.note.Note A>
    {1.9583} <music21.note.Note A>
    {2.4583} <music21.note.Note G#>
    {3.5187} <music21.note.Note F#>
    {3.7687} <music21.note.Note E>
    {4.0187} <music21.note.Note C#>
    {4.2687} <music21.note.Note E>
    {4.5187} <music21.note.Note F#>
    {4.7792} <music21.note.Note E>
    {5.0187} <music21.note.Note C#>
    {5.2687} <music21.note.Note E>
    {5.5187} <music21.note.Note F#>
    {5.7687} <music21.note.Note E>
    {6.0292} <music21.note.Note C#>
    {6.2687} <music21.note.Note B>
    {6.5187} <music21.note.Note C#>
    {6.7687} <music21.note.Note B>
    {7.0604} <music21.note.Note C#>
    {7.2687} <music21.note.Note F#>
    {7.5187} <music21.note.Note E>
    {2.9583} <music21.note.Note C#>


In [30]:
mel2.update_sequence_from_adaptation_data(ad_mel2)
mel2.sequence.instruments[0].notes
mel2.sequence.resolution

480

In [31]:
result = a.adapt(mel2, mel1)
print(result.meta)
result.sequence.instruments[0].notes

{'adaptation_steps': [{'name': 'TransposeSequenceOperation', 'duration': 0.2129976749420166, 'intermediate_result': <music21.stream.Score 0x163670b5948>}], 'adaptation_duration': 0.21399569511413574}


[Note(start=0.000000, end=0.260417, pitch=55, velocity=77),
 Note(start=0.354167, end=0.401042, pitch=55, velocity=77),
 Note(start=0.604167, end=0.645833, pitch=55, velocity=77),
 Note(start=0.854167, end=0.906250, pitch=55, velocity=77),
 Note(start=0.979167, end=1.052083, pitch=55, velocity=77),
 Note(start=1.229167, end=1.296875, pitch=54, velocity=77),
 Note(start=1.759375, end=1.926042, pitch=76, velocity=98),
 Note(start=1.884375, end=2.019792, pitch=74, velocity=90),
 Note(start=2.009375, end=2.155208, pitch=71, velocity=82),
 Note(start=2.134375, end=2.254167, pitch=74, velocity=86),
 Note(start=2.259375, end=2.431250, pitch=76, velocity=94),
 Note(start=2.389583, end=2.519792, pitch=74, velocity=86),
 Note(start=2.509375, end=2.603125, pitch=71, velocity=86),
 Note(start=2.634375, end=2.759375, pitch=74, velocity=90),
 Note(start=2.759375, end=2.946875, pitch=76, velocity=94),
 Note(start=2.884375, end=3.004167, pitch=74, velocity=86),
 Note(start=3.014583, end=3.139583, pitc